In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time
from pandas.plotting import table 
import matplotlib.pyplot as plt
import string

# Borrowed the offset idea from this (much more professionally done) tableau page
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet
# No adjustments for similar names etc (apologies)

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html']+['https://www.janestreet.com/puzzles/archive/page'+str(i)+'/index.html' for i in range(2,13)]
links =  ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [4]:
outputs = []
for newcomer_offset in [7]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df['Name']=df['Name'].str.lstrip("1234567890. ")
    df['Name'] = df['Name'].str.title()
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
   
df2.sort_values(['Score','Solved'],ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Evan Semet,43,44,0.843137
Karl Mahlburg,61,68,0.813333
Senthil Rajasekaran,67,79,0.779070
Gareth Owen,49,59,0.742424
Aaditya Raghavan,51,62,0.739130
Lazar Ilic,46,56,0.730159
Danica Xiong,24,26,0.727273
Blaine Hill,22,24,0.709677
Calvin Pozderac,70,92,0.707071


In [5]:
df2.sort_values('Solved',ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Calvin Pozderac,70,92,0.707071
Senthil Rajasekaran,67,79,0.779070
Karl Mahlburg,61,68,0.813333
Sean Egan,54,86,0.580645
Keith Schneider,51,83,0.566667
Aaditya Raghavan,51,62,0.739130
Gareth Owen,49,59,0.742424
Lazar Ilic,46,56,0.730159
Evan Semet,43,44,0.843137


In [6]:
import xlwings as xw
sht = xw.Book().sheets[0]
sht.range('a1').options(index=False).value =  df

In [7]:
#links